# Pashto Tokenization Notebook

This notebook demonstrates whitespace vs regex-based tokenization for Urdu text using the column **News Text**.

In [1]:
!pip install transformers

In [4]:
import pandas as pd
import re
import sentencepiece as spm
import os
from transformers import AutoTokenizer

In [3]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!pip install ./indic_nlp_library


Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1404, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 1404 (delta 139), reused 152 (delta 124), pack-reused 1219 (from 1)
Receiving objects: 100% (1404/1404), 9.57 MiB | 10.79 MiB/s, done.
Resolving deltas: 100% (749/749), done.
Processing ./indic_nlp_library
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 10.0 MB/s eta 0:00:00
  Created wheel for indic_nlp_library: filename=indic_nlp_library-0.92-py3-none-any.whl size=40778 sha256=317e8ead2b3502c60d72c513e7eed909c04ba546a15ac7dd6106b7f5c5c7ad91
  Stored in directory: /tmp/pip-ephem-wheel-cache-muvc9iq4/wheels/9a/20/c2/909246e3fb53bf4c760f1ada72e462b85e3588ed01d1ae2169
Successfully built indic_nlp_library


In [4]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git /content/indic_nlp_resources

Cloning into '/content/indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 31.82 MiB/s, done.
Resolving deltas: 100% (53/53), done.
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

^C


In [5]:
from indicnlp.tokenize import indic_tokenize

In [2]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **PASHTO Text Tokenization**

In [5]:
file_path_pashto = '/content/drive/MyDrive/Colab Notebooks/dataset/pashto_dataset.csv'

try:
    df_pashto = pd.read_csv(file_path_pashto)
    print("Pashto dataset loaded successfully:")
    print(df_pashto.head())
except FileNotFoundError:
    print(f"Error: The Pashto file at '{file_path_pashto}' was not found.")
except Exception as e:
    print(f"An error occurred while loading the Pashto dataset: {e}")

Pashto dataset loaded successfully:
                                                desc
0  له دې اعلان وروسته د دغه هېواد لر او بر لاریون...
1  د بریتانیا لسټر سیمې محکمې ټېک ټاکرې مهک بخاري...
2  د پاکستان د پوځ مشر وايي، چې هېواد به یې تر هغ...
3  د (هرډوز) کمپنۍ مشر محمد الفائد چې زوی یې (دود...
4  د طالبانو حکومت د کورنیو چارو وزارت کابل ښار ک...


In [ ]:

print(df_pashto.columns)

## Whitespace Tokenization (Not Recommended)

In [6]:
df_pashto['whitespace_tokens'] = df_pashto['desc'].astype(str).apply(lambda x: x.split())
df_pashto[['desc', 'whitespace_tokens']].head()

,desc,whitespace_tokens
0,له دې اعلان وروسته د دغه هېواد لر او بر لاریون...,"[له, دې, اعلان, وروسته, د, دغه, هېواد, لر, او,..."
1,د بریتانیا لسټر سیمې محکمې ټېک ټاکرې مهک بخاري...,"[د, بریتانیا, لسټر, سیمې, محکمې, ټېک, ټاکرې, م..."
2,د پاکستان د پوځ مشر وايي، چې هېواد به یې تر هغ...,"[د, پاکستان, د, پوځ, مشر, وايي،, چې, هېواد, به..."
3,د (هرډوز) کمپنۍ مشر محمد الفائد چې زوی یې (دود...,"[د, (هرډوز), کمپنۍ, مشر, محمد, الفائد, چې, زوی..."
4,د طالبانو حکومت د کورنیو چارو وزارت کابل ښار ک...,"[د, طالبانو, حکومت, د, کورنیو, چارو, وزارت, کا..."


## Regex-based Unicode Tokenization (Recommended)

In [7]:
def regex_tokenize_pashto(text):
    pattern = r'[\u0600-\u06FF]+'
    return re.findall(pattern, str(text))

df_pashto['regex_tokens'] = df_pashto['desc'].apply(regex_tokenize_pashto)
df_pashto[['desc', 'regex_tokens']].head()

,desc,regex_tokens
0,له دې اعلان وروسته د دغه هېواد لر او بر لاریون...,"[له, دې, اعلان, وروسته, د, دغه, هېواد, لر, او,..."
1,د بریتانیا لسټر سیمې محکمې ټېک ټاکرې مهک بخاري...,"[د, بریتانیا, لسټر, سیمې, محکمې, ټېک, ټاکرې, م..."
2,د پاکستان د پوځ مشر وايي، چې هېواد به یې تر هغ...,"[د, پاکستان, د, پوځ, مشر, وايي،, چې, هېواد, به..."
3,د (هرډوز) کمپنۍ مشر محمد الفائد چې زوی یې (دود...,"[د, هرډوز, کمپنۍ, مشر, محمد, الفائد, چې, زوی, ..."
4,د طالبانو حکومت د کورنیو چارو وزارت کابل ښار ک...,"[د, طالبانو, حکومت, د, کورنیو, چارو, وزارت, کا..."


# **Subword tokenization using SentencePiece**

In [8]:
# Step 1: Save all text to a temporary file
corpus_file = '/content/drive/MyDrive/Colab Notebooks/dataset/pashto_corpus.txt'
with open(corpus_file, 'w', encoding='utf-8') as f:
    for text in df_pashto['desc']:
        f.write(str(text) + '\n')

# Step 2: Train SentencePiece model
spm.SentencePieceTrainer.train(
    input=corpus_file,
    model_prefix='/content/drive/MyDrive/Colab Notebooks/dataset/pashto_spm',
    vocab_size=8000,  # you can increase for larger datasets
    character_coverage=1.0,
    model_type='unigram'  # options: unigram, bpe, char
)

# Step 3: Load the trained model
sp = spm.SentencePieceProcessor(model_file='/content/drive/MyDrive/Colab Notebooks/dataset/pashto_spm.model')

# Step 4: Apply subword tokenization
df_pashto['subword_tokens'] = df_pashto['desc'].astype(str).apply(lambda x: sp.encode(x, out_type=str))


# Tokenize Urdu using BERT tokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
df_pashto['bert_encoding'] = df_pashto['desc'].astype(str).apply(
    lambda x: tokenizer(
        x,
        truncation=True,
        max_length=512,
        padding=False,
        return_attention_mask=False,
        return_token_type_ids=False
    )
)

# Extract tokens safely
df_pashto['bert_tokens'] = df_pashto['bert_encoding'].apply(lambda x: tokenizer.convert_ids_to_tokens(x['input_ids']))


# Calculate token lengths

In [12]:
df_pashto['whitespace_len'] = df_pashto['whitespace_tokens'].apply(len)
df_pashto['regex_len'] = df_pashto['regex_tokens'].apply(len)
df_pashto['subword_len'] = df_pashto['subword_tokens'].apply(len)
df_pashto['bert_len'] = df_pashto['bert_tokens'].apply(len)

**Preview Result**

In [13]:
df_pashto[['desc', 'whitespace_tokens', 'regex_tokens', 'subword_tokens', 'bert_tokens']].head()

,desc,whitespace_tokens,regex_tokens,subword_tokens,bert_tokens
0,له دې اعلان وروسته د دغه هېواد لر او بر لاریون...,"[له, دې, اعلان, وروسته, د, دغه, هېواد, لر, او,...","[له, دې, اعلان, وروسته, د, دغه, هېواد, لر, او,...","[▁له, ▁دې, ▁اعلان, ▁وروسته, ▁د, ▁دغه, ▁هېواد, ...","[[CLS], له, [UNK], ا, ##علان, و, ##روس, ##ته, ..."
1,د بریتانیا لسټر سیمې محکمې ټېک ټاکرې مهک بخاري...,"[د, بریتانیا, لسټر, سیمې, محکمې, ټېک, ټاکرې, م...","[د, بریتانیا, لسټر, سیمې, محکمې, ټېک, ټاکرې, م...","[▁د, ▁بریتانیا, ▁لس, ټر, ▁سیمې, ▁محکمې, ▁ټېک, ...","[[CLS], د, بریتانیا, [UNK], [UNK], [UNK], [UNK..."
2,د پاکستان د پوځ مشر وايي، چې هېواد به یې تر هغ...,"[د, پاکستان, د, پوځ, مشر, وايي،, چې, هېواد, به...","[د, پاکستان, د, پوځ, مشر, وايي،, چې, هېواد, به...","[▁د, ▁پاکستان, ▁د, ▁پوځ, ▁مشر, ▁وايي, ،, ▁چې, ...","[[CLS], د, پاکستان, د, [UNK], م, ##شر, واي, ##..."
3,د (هرډوز) کمپنۍ مشر محمد الفائد چې زوی یې (دود...,"[د, (هرډوز), کمپنۍ, مشر, محمد, الفائد, چې, زوی...","[د, هرډوز, کمپنۍ, مشر, محمد, الفائد, چې, زوی, ...","[▁د, ▁(, ه, ر, ډو, ز, ), ▁کمپنۍ, ▁مشر, ▁محمد, ...","[[CLS], د, (, [UNK], ), [UNK], م, ##شر, محمد, ..."
4,د طالبانو حکومت د کورنیو چارو وزارت کابل ښار ک...,"[د, طالبانو, حکومت, د, کورنیو, چارو, وزارت, کا...","[د, طالبانو, حکومت, د, کورنیو, چارو, وزارت, کا...","[▁د, ▁طالبانو, ▁حکومت, ▁د, ▁کورنیو, ▁چارو, ▁وز...","[[CLS], د, طالب, ##انو, حکومت, د, کو, ##رن, ##..."


In [14]:
df_pashto[['desc',  'whitespace_len', 'regex_len', 'subword_len', 'bert_len']].head()


,desc,whitespace_len,regex_len,subword_len,bert_len
0,له دې اعلان وروسته د دغه هېواد لر او بر لاریون...,81,83,106,145
1,د بریتانیا لسټر سیمې محکمې ټېک ټاکرې مهک بخاري...,645,657,926,512
2,د پاکستان د پوځ مشر وايي، چې هېواد به یې تر هغ...,358,367,463,512
3,د (هرډوز) کمپنۍ مشر محمد الفائد چې زوی یې (دود...,431,447,628,512
4,د طالبانو حکومت د کورنیو چارو وزارت کابل ښار ک...,337,348,447,512


## Save Tokenized Dataset

In [15]:
# -------------------------------
# 6. Save tokenized dataset
# -------------------------------
output_path_pashto = '/content/drive/MyDrive/Colab Notebooks/dataset/pashto_dataset_tokenized.xlsx'
df_pashto.to_excel(output_path_pashto, index=False)
print("Pashto dataset with subword tokenization saved successfully!")

Pashto dataset with subword tokenization saved successfully!
